# Quantum Fourier Transform Benchmark

In [1]:
low = 8
high = 64
samples = 10

In [2]:
import time
import random
import math

from pyqrack import QrackSimulator

## PyQrack

In [3]:
def bench_qrack(n):
    # This is basically a "quantum volume" (random) circuit.
    start = time.perf_counter()

    sim = QrackSimulator(n)

    lcv_range = range(n)
    all_bits = [list(lcv_range)]

    for d in lcv_range:
        # Single-qubit gates
        for i in lcv_range:
            sim.u(i, random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi), random.uniform(0, 2 * math.pi))

        # 2-qubit couplers
        unused_bits = all_bits.copy()
        random.shuffle(unused_bits)
        while len(unused_bits) > 1:
            sim.mcx([unused_bits.pop()], unused_bits.pop())

    # Terminal measurement
    sim.m_all()

    return time.perf_counter() - start

# Make sure the OpenCL environment is initialized before timing.
# (You probably also want to precompile OpenCL kernels with the `qrack_cl_compile` utility, in general.)
bench_qrack(1)

qrack_results = {}
for n in range(low, high + 1):
    width_results = []
        
    # Run the benchmarks
    for i in range(samples):
        width_results.append(bench_qrack(n))

    qrack_results[n] = sum(width_results) / samples
    print(n, ": ", qrack_results[n], " seconds")

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
8 :  0.00025167620005959177  seconds
9 :  0.00031617259955964985  seconds
10 :  0.00043385910030337984  seconds
11 :  0.0004521234004641883  seconds
12 :  0.0005281858000671491  seconds
13 :  0.0006256255004700506  seconds
14 :  0.0007098271005816059  seconds
15 :  0.0007807215006323531  seconds
16 :  0.0008685215994773898  seconds
17 :  0.0009965575001842808  seconds
18 :  0.0011396573998354142  seconds
19 :  0.0012694912002189085  seconds
20 :  0.0014295133008999984  seconds
21 :  0.0015742867995868436  seconds
22 :  0.0017161945001134883  seconds
23 :  0.001893810599722201  seconds
24 :  0.0020371959999465616  seconds
25 :  0.00217880439995497  seconds
26 :  0.002387791299406672  seconds
27 :  0.00255318220006302  seconds
28 :  0.003321845600294182  seconds
29 :  0.003533299599439488  seconds
30 :  0.003826791099345428  seconds
31 :  0.004028105599718401  seconds
32 :  0.004322662199